### Using Ragas to evaluate our RAG pipeline outpus

In [2]:
import os, openai
from langsmith import Client
from qdrant_client import QdrantClient

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

### Download an example reference dataset from Langsmwith

In [6]:
client = Client()
dataset = client.read_dataset(
    dataset_name="rag-evaluation-dataset",
)




In [7]:
dataset

Dataset(name='rag-evaluation-dataset', description='A dataset of synthetic questions for RAG evaluation', data_type=<DataType.kv: 'kv'>, id=UUID('22b6684e-e86a-41e4-933c-a0e92b1fdba3'), created_at=datetime.datetime(2026, 1, 17, 16, 13, 49, 685925, tzinfo=TzInfo(0)), modified_at=datetime.datetime(2026, 1, 17, 16, 13, 49, 685925, tzinfo=TzInfo(0)), example_count=39, session_count=0, last_session_start_time=None, inputs_schema=None, outputs_schema=None, transformations=None, metadata={'runtime': {'sdk': 'langsmith-py', 'library': 'langsmith', 'runtime': 'python', 'platform': 'macOS-15.6.1-arm64-arm-64bit', 'sdk_version': '0.6.4', 'runtime_version': '3.12.12', 'langchain_version': None, 'py_implementation': 'CPython', 'langchain_core_version': None}})

In [9]:
list(client.list_examples(dataset_id=dataset.id))[0].outputs

{'ground_truth': 'Wholesale pricing or bulk discount information is not provided in the product description; that information is not available in the given context.',
 'reference_chunks': ['B08P1T8R46'],
 'reference_descriptions': ["SD Card Reader, uni High-Speed USB 3.0 to Micro SD Card Adapter, Aluminum Computer Memory Card Reader Dual Slots, for SD/SDXC/SDHC/MMC/Micro SDXC/TF/Micro SDHC | Windows, Linux, macOS, and more ['[ULTRA FAST DATA TRANSFER] - USB 3.0 brings you fast and reliable 5Gbps photo, video transfer from camera to laptop, reducing transfer time and improving your work efficiency. Backward compatible with USB 2.0 & 1.1.', '[PLUG & PLAY] - No additional software or driver required, also fully powered by your USB port, no additional power supply needed. (!! This USB SD card adapter reads UHS-II memory cards at UHS-I speed).', '[READ & WRITE SIMULTANEOUSLY] - uni USB to SD Card Adapter can read and write 2 cards at the same time, transfer your materials without transit an

In [11]:
# the reference question
list(client.list_examples(dataset_id=dataset.id))[0].inputs

{'question': 'Unanswerable: What is the wholesale discount for ordering 100 units of the uni USB 3.0 card reader (B08P1T8R46)?'}

In [14]:
reference_input = list(client.list_examples(dataset_id=dataset.id))[0].inputs
reference_output = list(client.list_examples(dataset_id=dataset.id))[0].outputs

### Read the RAG endpoint to evaluate

In [15]:
# the end point is http://localhost:8000/ 
# first get the rag retrived data for a sample question
import requests
question = reference_input["question"]
response = requests.post(
    url="http://localhost:8000/product_assistant",
    json={"query": question},
)
rag_output = response.json()



In [19]:
sample_output = rag_output["response"]
sample_output

{'question': 'Unanswerable: What is the wholesale discount for ordering 100 units of the uni USB 3.0 card reader (B08P1T8R46)?',
 'answer': 'I don’t have wholesale pricing or bulk-discount information for ordering 100 units of the uni USB 3.0 card reader (B08P1T8R46) in our current inventory. I can provide the product specifications or help with other details if you’d like.',
 'retrieved_context_ids': ['B08P1T8R46',
  'B09Q61G298',
  'B0BR9135RC',
  'B0BW5T1F9J',
  'B09VGLGKM4'],
 'retrieved_context': ["SD Card Reader, uni High-Speed USB 3.0 to Micro SD Card Adapter, Aluminum Computer Memory Card Reader Dual Slots, for SD/SDXC/SDHC/MMC/Micro SDXC/TF/Micro SDHC | Windows, Linux, macOS, and more ['[ULTRA FAST DATA TRANSFER] - USB 3.0 brings you fast and reliable 5Gbps photo, video transfer from camera to laptop, reducing transfer time and improving your work efficiency. Backward compatible with USB 2.0 & 1.1.', '[PLUG & PLAY] - No additional software or driver required, also fully powere

### Basic RAGAS metrics

In [ ]:
from openai import AsyncOpenAI
from ragas.metrics.collections import Faithfulness
from ragas.llms import llm_factory

client = AsyncOpenAI()
llm = llm_factory(model="gpt-4o-mini", client=client)

score = Faithfulness(llm=llm)

result = await score.ascore(
    user_input=sample_output["question"],
    response=sample_output["answer"],
    retrieved_contexts=sample_output["retrieved_context"],
)
print("Faithfulness score: ", result.value)

Faithfulness score:  0.6666666666666666


In [ ]:
# ragas response relevancy
from ragas.metrics.collections import AnswerRelevancy
from ragas.embeddings import OpenAIEmbeddings

client = AsyncOpenAI()
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", client=client)
llm = llm_factory(model="gpt-4o-mini", client=client)

score = AnswerRelevancy(llm=llm, embeddings=embeddings)

result = await score.ascore(
    user_input=sample_output["question"],
    response=sample_output["answer"],
)
print("Response relevancy score: ", result.value)



Response relevancy score:  0.0


In [ ]:
#ID based context precision and recall
from ragas.metrics import IDBasedContextPrecision
from ragas import SingleTurnSample

score = IDBasedContextPrecision()

sample = SingleTurnSample(
    retrieved_context_ids=sample_output["retrieved_context_ids"],
    reference_context_ids=reference_output["reference_chunks"],
)

result = score.single_turn_score(sample)
print("ID based context precision: ", result)


ID based context precision:  0.2


/var/folders/qh/p49skmb55cg_tkny7j9ybm7h0000gn/T/ipykernel_52128/681393904.py:2: DeprecationWarning: Importing IDBasedContextPrecision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import IDBasedContextPrecision
  from ragas.metrics import IDBasedContextPrecision


In [50]:
#ID based context recall
from ragas.metrics import IDBasedContextRecall

score = IDBasedContextRecall()

sample = SingleTurnSample(
    retrieved_context_ids=sample_output["retrieved_context_ids"],
    reference_context_ids=reference_output["reference_chunks"],
)
result = score.single_turn_score(sample)
print("ID based context recall: ", result)

ID based context recall:  1.0


/var/folders/qh/p49skmb55cg_tkny7j9ybm7h0000gn/T/ipykernel_52128/2936492673.py:2: DeprecationWarning: Importing IDBasedContextRecall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import IDBasedContextRecall
  from ragas.metrics import IDBasedContextRecall
